In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

service = QiskitRuntimeService()
backend = service.backend("ibm_fez")
iris = datasets.load_iris()
X = iris.data
y = iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_hw = X_scaled[:3]
y_hw = y[:3]
n_qubits = 4
shots = 64
dev = qml.device(
    "qiskit.remote",
    wires=n_qubits,
    backend=backend,
    shots=shots,
)

def feature_map(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

def param_layer(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)

@qml.qnode(dev)
def quantum_embedding(x, params):
    feature_map(x)
    param_layer(params)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

np.random.seed(0)
params = np.random.uniform(low=-0.5, high=0.5, size=(n_qubits,))
embs = []
for x in X_hw:
    e = quantum_embedding(x, params)
    embs.append(e)
embs = np.array(embs, dtype=float)

print("Labels (3 samples):", y_hw)
print("Quantum embeddings from QPU:")
print(embs)


qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-11-15 04:14:40,711: Default instance not set. Searching all available instances.
/usr/local/python/3.12.1/lib/python3.12/site-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


Labels (3 samples): [0 0 0]
Quantum embeddings from hardware:
[[0.47563015 0.12661515 0.3151739  0.25338695]
 [0.44389398 1.01179522 0.50399902 0.60186847]
 [0.12680329 0.85549848 0.40904621 0.09489682]]
